In [288]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('max_columns', None)
import itertools
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [181]:
#importing dataset
nl_set = pd.read_csv('/Users/romanpeerboom/Documents/Thesis/Basic_Patstat/PATSAT_NL.csv', delimiter = ';')

In [182]:
ger1 = pd.read_csv('/Users/romanpeerboom/Documents/Thesis/Basic_Patstat/GER_1.csv', delimiter = ';')
ger2 = pd.read_csv('/Users/romanpeerboom/Documents/Thesis/Basic_Patstat/GER_2.csv', delimiter = ';')
gbfr = pd.read_csv('/Users/romanpeerboom/Documents/Thesis/Basic_Patstat/GB_FR.csv', delimiter = ';')
rest = pd.read_csv('/Users/romanpeerboom/Documents/Thesis/Basic_Patstat/rest.csv', delimiter = ';')
nl_set = pd.read_csv('/Users/romanpeerboom/Documents/Thesis/Basic_Patstat/PATSAT_NL.csv', delimiter = ';')

In [183]:
#creating one big set with a new column with 4 digits of ipc class
frames = [ger1, ger2, gbfr, rest]
total_set = pd.concat(frames, ignore_index = True)
classes = total_set['ipc_class_symbol'].tolist()
classes2 = [w[:4] for w in classes]
total_set['ipc_class'] = classes2
total_set = total_set.rename(columns={"Unnamed: 1": "appln_auth"})

In [184]:
patents = total_set['appln_auth'].unique().tolist()
patents = patents

In [185]:
#creating a list of lists in which each of the lists is all the ipc codes that belong to one patent
#only selecting the ones in which multiple ipc classes appear because others not relevant
all_patent_codes = []
for patent in tqdm(patents):
    patent_set = total_set.loc[total_set['appln_auth'] == patent]
    patent_codes = patent_set['ipc_class'].unique().tolist()
    if len(patent_codes) > 1:
        all_patent_codes.append(patent_codes)
print(all_patent_codes)

100%|██████████| 391286/391286 [22:45:20<00:00,  4.78it/s]       


[['H02J', 'H02P'], ['H02J', 'H02M'], ['H02G', 'H02H'], ['H02J', 'H02M'], ['H02J', 'H02M'], ['H02G', 'H02K'], ['H02G', 'H02M'], ['H02H', 'H02M'], ['H02H', 'H02P'], ['H02H', 'H02P'], ['H02H', 'H02P'], ['H02H', 'H02P'], ['H02H', 'H02M'], ['H02M', 'H02P'], ['H02J', 'H02P'], ['H02H', 'H02P'], ['H02J', 'H02P'], ['H02J', 'H02M'], ['H02H', 'H02P'], ['H02J', 'H02P'], ['H02H', 'H02P'], ['H02B', 'H02G'], ['H02B', 'H02G'], ['H02K', 'H02P'], ['H02K', 'H02P'], ['H02K', 'H02P'], ['H02J', 'H02M'], ['H02K', 'H02P'], ['H02H', 'H02M'], ['H02K', 'H02P'], ['H02J', 'H02M'], ['H02H', 'H02J'], ['H02J', 'H02M'], ['H02J', 'H02P'], ['H02J', 'H02M'], ['H02G', 'H02H'], ['H02B', 'H02H', 'H02J'], ['H02H', 'H02J'], ['H02H', 'H02J'], ['H02M', 'H02P'], ['H02H', 'H02M'], ['H02M', 'H02P'], ['H02H', 'H02M'], ['H02M', 'H02P'], ['H02B', 'H02G'], ['H02B', 'H02G'], ['H02B', 'H02G'], ['H02B', 'H02G'], ['H02J', 'H02M'], ['H02J', 'H02P'], ['H02B', 'H02G'], ['H02B', 'H02M'], ['H02B', 'H02G'], ['H02M', 'H02P'], ['H02K', 'H02P'], [

In [269]:
def co_matrix(allowed_words, documents):
    word_to_id = dict(zip(allowed_words, range(len(allowed_words))))
    documents_as_ids = [np.sort([word_to_id[w] for w in doc if w in word_to_id]).astype('uint32') for doc in documents]
    row_ind, col_ind = zip(*itertools.chain(*[[(i, w) for w in doc] for i, doc in enumerate(documents_as_ids)]))
    data = np.ones(len(row_ind), dtype='uint32')  # use unsigned int for better memory utilization
    max_word_id = max(itertools.chain(*documents_as_ids)) + 1
    docs_words_matrix = scipy.sparse.csr_matrix((data, (row_ind, col_ind)), shape=(len(documents_as_ids), max_word_id))  # efficient arithmetic operations with CSR * CSR
    words_cooc_matrix = docs_words_matrix.T * docs_words_matrix  # multiplying docs_words_matrix with its transpose matrix would generate the co-occurences matrix
    words_cooc_matrix.setdiag(0)
    print(f"words_cooc_matrix:\n{words_cooc_matrix.todense()}")
    return words_cooc_matrix, word_to_id 


In [281]:
#this is the result
codes = total_set['ipc_class'].unique().tolist()
matrix, word_to_id = co_matrix(codes, all_patent_codes)
co = pd.DataFrame(data=scipy.sparse.csr_matrix.todense(matrix), columns = codes, index = codes)
co.to_csv(r'/Users/romanpeerboom/Documents/Thesis/Matrix.csv', index = True, sep = ';')

words_cooc_matrix:
[[   0  111  599  354  225 3375  125  272   68]
 [ 111    0 3468 5616 4653   63   68 2714    9]
 [ 599 3468    0 1584 9018  502  398 3998  587]
 [ 354 5616 1584    0 1041  143  668  860   51]
 [ 225 4653 9018 1041    0  218  190 3473  148]
 [3375   63  502  143  218    0   17  446   15]
 [ 125   68  398  668  190   17    0   82  145]
 [ 272 2714 3998  860 3473  446   82    0  115]
 [  68    9  587   51  148   15  145  115    0]]


In [282]:
co

,H02G,H02P,H02J,H02K,H02M,H02B,H02N,H02H,H02S
H02G,0,111,599,354,225,3375,125,272,68
H02P,111,0,3468,5616,4653,63,68,2714,9
H02J,599,3468,0,1584,9018,502,398,3998,587
H02K,354,5616,1584,0,1041,143,668,860,51
H02M,225,4653,9018,1041,0,218,190,3473,148
H02B,3375,63,502,143,218,0,17,446,15
H02N,125,68,398,668,190,17,0,82,145
H02H,272,2714,3998,860,3473,446,82,0,115
H02S,68,9,587,51,148,15,145,115,0


In [296]:
similarities = cosine_similarity(co)
frame = pd.DataFrame(similarities, columns = codes, index = codes)
frame

,H02G,H02P,H02J,H02K,H02M,H02B,H02N,H02H,H02S
H02G,1.000000,0.205449,0.158577,0.124599,0.213910,0.044419,0.213496,0.253352,0.227402
H02P,0.205449,1.000000,0.682832,0.248088,0.507172,0.174926,0.884316,0.678855,0.613710
H02J,0.158577,0.682832,1.000000,0.512333,0.279894,0.169858,0.397151,0.663656,0.296340
H02K,0.124599,0.248088,0.512333,1.000000,0.670302,0.142314,0.266272,0.697041,0.351540
H02M,0.213910,0.507172,0.279894,0.670302,1.000000,0.195670,0.551800,0.764533,0.846758
H02B,0.044419,0.174926,0.169858,0.142314,0.195670,1.000000,0.279795,0.190965,0.280241
H02N,0.213496,0.884316,0.397151,0.266272,0.551800,0.279795,1.000000,0.612322,0.593191
H02H,0.253352,0.678855,0.663656,0.697041,0.764533,0.190965,0.612322,1.000000,0.768991
H02S,0.227402,0.613710,0.296340,0.351540,0.846758,0.280241,0.593191,0.768991,1.000000
